<a href="https://colab.research.google.com/github/dmitriygorlov/Yandex.Practikum_Data_Science/blob/main/Module-03_04-Machine-learning-for-texts/project-12_text-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект для «Викишоп»

Нужно обучить модель классифицировать комментарии на позитивные и негативные.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Библиотеки-и-первичное-прочтение-файла" data-toc-modified-id="Библиотеки-и-первичное-прочтение-файла-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Библиотеки и первичное прочтение файла</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBM</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

### Библиотеки и первичное прочтение файла

In [1]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


загрузили библиотеки

In [2]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-09-05 19:52:42--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.243, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-09-05 19:52:42--  http://cache-mskm903.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)... 5.45.220.13, 2a02:6b8:0:2002::14
Connecting to cache-mskm903.cdn.yandex.net (cache-mskm903.cdn.yandex.net)|5.45.220.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

**специально загрузил версию младше, со старшей вешался google colab на лемматизации**

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

загрузили файл, посмотрим ближе

In [4]:
display(df.head())
df.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df.sample(n = 5, random_state = 12345)

,text,toxic
146790,Ahh shut the fuck up you douchebag sand nigger...,1
2941,"""\n\nREPLY: There is no such thing as Texas Co...",0
115087,"Reply\nHey, you could at least mention Jasenov...",0
48830,"Thats fine, there is no deadline ) chi?",0
136034,"""\n\nDYK nomination of Mustarabim\n Hello! You...",0


In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Файл достаточно большой, содержит почти 160 000 строк, важно учитывать при тяжких задачах. При этом у нас классическая задача классификации текста. Есть признак text, который необходимо подготовить, а также у нас есть целевой признак toxic с большим перекосом значений (9 к 1), что может быть проблемой для достижения заданной f-меры. Для начала разберёмся со столбцом text.

### Подготовка признаков

Подготовим функции и создадим новый столбец

In [7]:
# df['text'] = df['text'].values.astype('U')

In [8]:
# m = Mystem()

# def clear_lemm_text(text):
#     filtro = re.sub(r'[^a-zA-Z]', ' ', text)
#     splito = filtro.split()
#     clearo = ' '.join(splito)
#     lemm_list = m.lemmatize(clearo)
#     lemm_text = ' '.join(lemm_list)
#     return lemm_text

In [9]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def clear_lemm_text_word(text):
    filtro = re.sub(r'[^a-zA-Z]', ' ', str(text))
    splito = filtro.split()
    # clearo = ' '.join(splito)
    lemm_list = list()
    for w in nltk.word_tokenize(' '.join(splito)):
        lemmm_uno = lemmatizer.lemmatize(w, get_wordnet_pos(w))
        lemm_list.append(lemmm_uno)
    lemm_text = ' '.join(lemm_list)
    return lemm_text

In [10]:
clifff = df['text'].head().apply(clear_lemm_text_word)
print(clifff)

0    Explanation Why the edits make under my userna...
1    D aww He match this background colour I m seem...
2    Hey man I m really not try to edit war It s ju...
3    More I can t make any real suggestion on impro...
4    You sir be my hero Any chance you remember wha...
Name: text, dtype: object


In [11]:
# df['text'] = df['text'].astype(str)
df['ready_word_text'] = df['text'].apply(clear_lemm_text_word)
# df['text'].head().apply(clear_lemm_text_word)

In [12]:
df.head()


,text,toxic,ready_word_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


In [13]:
# import nltk
# nltk.download('averaged_perceptron_tagger')z

теперь пройдёмся по столбцу и создадим новый (очищенный и лемматизированны), готовый к использованию

In [14]:
# df['ready_text'] = df['text'].apply(clear_lemm_text)

теперь поделим на обучающую и тестовую части

In [15]:
features = df['ready_word_text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.25, random_state = 12345, stratify = target)

подготовка готова, перейдём к обучению

## Обучение

Воспользуемся 2 моделями:

- Линейная регрессия
- LGBM

(Тут было двухчасовое мучение с Catboost и google colab, но сражение было проиграно)

Для начала создадим признаки с помощью TF-IDF

In [16]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2)) 

tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

print('Размер train:', tf_idf_train.shape)
print('Размер test:', tf_idf_test.shape)

Размер train: (119678, 2019902)
Размер test: (39893, 2019902)
CPU times: user 28.6 s, sys: 659 ms, total: 29.3 s
Wall time: 29.2 s


Признаки созданы, перейдём к моделям



### Линейная регрессия

Для начала посмотрим на необходимость балансирования весов классов. Запустим 2 версии модели 

In [17]:
%%time

model_LR = LogisticRegression()
train_f1 = cross_val_score(model_LR, 
                      tf_idf_train, 
                      target_train, 
                      cv=3, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

F1 на CV 0.5874094315963835
CPU times: user 1min 20s, sys: 42.3 s, total: 2min 2s
Wall time: 1min 9s


In [18]:
%%time

model_LR_balanced = LogisticRegression(class_weight='balanced')
train_f1 = cross_val_score(model_LR_balanced, 
                      tf_idf_train, 
                      target_train, 
                      cv=3, 
                      scoring='f1').mean()
print('F1 на CV', train_f1)

F1 на CV 0.7500256181840493
CPU times: user 1min 10s, sys: 37.8 s, total: 1min 48s
Wall time: 1min 1s


Невооруженным взглядом видно, насколько баланс важен. Будем использовать только эту модель. На основе неё запустим перебор гиперпараметров

In [19]:
%%time

lr = LogisticRegression(max_iter = 1000, class_weight='balanced')
lr_params = {'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
             'C':[0.1, 1, 10]}

clf_lr = GridSearchCV(estimator = lr, param_grid = lr_params, scoring='f1', cv = 3)

clf_lr.fit(tf_idf_train, target_train)

print(clf_lr.best_params_)

{'C': 10, 'solver': 'newton-cg'}
CPU times: user 11min 41s, sys: 6min 24s, total: 18min 5s
Wall time: 10min 15s


Лучшие параметры найдены, посмотрим на получившийся F1 на тестовой выборке.

In [20]:
# clf_lr.fit(tf_idf_train, target_train)

lr_predict = clf_lr.predict(tf_idf_test)

print('F1 =', f1_score(target_test, lr_predict))

F1 = 0.7876344086021505


Несмотря на долгий подбор, модель Линейной регрессии справилась с заданным порогом метрики f1 в 0.75, показав результат в 0.787

### LGBM

Теперь попробуем LGBM

In [21]:
%%time
LGBM = LGBMClassifier(class_weight='balanced')
parametrs_LGBM = {'n_estimators' : [100],
              'depth' : [15, 5],
              'learning_rate' : [0.2],
              'random_state' : [12345]}
clf_LGBM = GridSearchCV(estimator = LGBM, param_grid = parametrs_LGBM, scoring='f1', cv = 3)

clf_LGBM.fit(tf_idf_train, target_train)

print(clf_LGBM.best_params_)

{'depth': 15, 'learning_rate': 0.2, 'n_estimators': 100, 'random_state': 12345}
CPU times: user 16min 2s, sys: 3.25 s, total: 16min 6s
Wall time: 8min 27s


Лучшие параметры найдены, посмотрим какую метрику покаже модель на тесте.

In [22]:
lgbm3 = LGBMClassifier(n_estimators = 100,
              depth = 15,
            learning_rate = 0.2,
              random_state = 12345)

lgbm3.fit(tf_idf_train, target_train)

LGBM3_predict = lgbm3.predict(tf_idf_test)

print('F1 =', f1_score(target_test, LGBM3_predict))

F1 = 0.7698478561549101


LGBM показал отличный результат по F1 в 0.769, что также побило требуемый уровень в 0.75

## Выводы

**По результатам исследования подобрана лучшая модель - это модель Линейной Регрессии с гиперпараметрами: max_iter = 1000, class_weight='balanced', C = 10, solver = lbfgs! . Она показала F1 выше требуемого (0.787)**

*Важный комментарий: в целевом признаке наблюдается сильный дисбаланс, положительных признаков (токсичных комментариев) всего лишь 10% от общего числа! В связи с этим проверка на адекватность не проводилась, требуемый показатель F1 достаточно определяет целесообразность модели*

В процессе исследования мы провели лемматизацию и очищение текстов, а также провели создание признаков с помощью TF-IDF, после чего протестировали 2 модели с разными гиперпараметрами.